# Setup
(no sé si hace falta con Jupyter pero lo dejo por las dudas, entiendo que sí)

This project uses Python version `3.8`.

From the project root directory, on a [virtual Python environment](https://virtualenvwrapper.readthedocs.io/en/latest/) (or not, if you're feeling brave), run:
```bash
pip3 install -r requirements.txt
```

Make sure that the source directory is added to your `$PYTHONPATH` environment variable.

In [124]:
import pandas as pd
import numpy as np

# Dataset
Dediqué bastante tiempo a buscar un dataset. Recuerdo que dijiste que "a partir de 50 variables es más divertido", pero principalmente encontré tres tipos de datasets:

* Datos listos para usar, con menos de 15 variables ya procesadas.
* Tablas gigantes con un montón de variables, llenas de valores inválidos y sin un target feature claro.
* Datasets de clasificación de imágenes.

Finalmente elegí [Rain in Australia](https://www.kaggle.com/datasets/jsphyg/weather-dataset-rattle-package), que si bien solo tiene dos clases, tiene 23 columnas de diversos tipos y requiere bastante preprocesamiento. Son datos meteorológicos de Australia con un target feature `RainTomorrow`, que responde a la pregunta "¿llovió al día siguiente?". Los datos están en `weatherAUS.csv`.

# Preprocesamiento de datos


In [125]:
data_frame = pd.read_csv("weatherAUS.csv")
data_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

A primera vista vemos que las columnas tienen distintos tipos (`float64` para valores numéricos, `object` para variables categóricas) y que faltan valores. Ignoro por qué el tipo de las categóricas es `object` y no `string`.

## Datos faltantes
La verdad no tengo idea de meteorología (ni del dataset en específico) y no tengo a quién preguntarle, así que no voy a rellenar datos. Mi criterio fue:
* Si falta `RainTomorrow`, borro la fila.
* Si falta más del 10% de los datos de una columna, elimino la variable.
* Si falta hasta el 10% de los datos de una columna, borro las filas donde no esté (total tengo muchos puntos).


In [126]:
data_frame = data_frame[data_frame['RainTomorrow'].notna()]
data_frame = data_frame.loc[:, data_frame.isnull().mean() < .1]
data_frame = data_frame.dropna()


## Variables numéricas TODO: Buscar Outliers




In [127]:
data_frame.select_dtypes("number")


,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm
0,13.4,22.9,0.6,44.0,20.0,24.0,71.0,22.0,1007.7,1007.1,16.9,21.8
1,7.4,25.1,0.0,44.0,4.0,22.0,44.0,25.0,1010.6,1007.8,17.2,24.3
2,12.9,25.7,0.0,46.0,19.0,26.0,38.0,30.0,1007.6,1008.7,21.0,23.2
3,9.2,28.0,0.0,24.0,11.0,9.0,45.0,16.0,1017.6,1012.8,18.1,26.5
4,17.5,32.3,1.0,41.0,7.0,20.0,82.0,33.0,1010.8,1006.0,17.8,29.7
...,...,...,...,...,...,...,...,...,...,...,...,...
145454,3.5,21.8,0.0,31.0,15.0,13.0,59.0,27.0,1024.7,1021.2,9.4,20.9
145455,2.8,23.4,0.0,31.0,13.0,11.0,51.0,24.0,1024.6,1020.3,10.1,22.4
145456,3.6,25.3,0.0,22.0,13.0,9.0,56.0,21.0,1023.5,1019.1,10.9,24.5
145457,5.4,26.9,0.0,37.0,9.0,9.0,53.0,24.0,1021.0,1016.8,12.5,26.1


## Variables categóricas

In [128]:
data_frame.select_dtypes("object")


,Date,Location,WindGustDir,WindDir9am,WindDir3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,W,W,WNW,No,No
1,2008-12-02,Albury,WNW,NNW,WSW,No,No
2,2008-12-03,Albury,WSW,W,WSW,No,No
3,2008-12-04,Albury,NE,SE,E,No,No
4,2008-12-05,Albury,W,ENE,NW,No,No
...,...,...,...,...,...,...,...
145454,2017-06-20,Uluru,E,ESE,E,No,No
145455,2017-06-21,Uluru,E,SE,ENE,No,No
145456,2017-06-22,Uluru,NNW,SE,N,No,No
145457,2017-06-23,Uluru,N,SE,WNW,No,No


### `Date`
Primero pasemos las fechas a `DateTime`. Luego voy a pecar un poco. Me importan dos cosas:
* Preservar distancias entre fechas de calendario (sin importar el año). Porque imagino que si todos los 31 de diciembre son muy calurosos, los 1 de enero también. Para esto hago una transformación sinusoidal (con la chanchada de considerar 366 días así no pienso en años bisiestos).
* Considerar los años (por si cada año llueve menos, por ejemplo). Para esto simplemente separo el año.

In [129]:
data_frame["Date"] = pd.to_datetime(data_frame["Date"])
data_frame["Year"] = data_frame["Date"].dt.year
data_frame["SinDate"] = np.sin(
    2 * np.pi * (data_frame["Date"].dt.dayofyear) / 366)


### Direcciones
Para las direcciones de viento, primero numero cada dirección empezando por N (que es 0) y continuando en sentido horario según:

<img src="compass.pbm" alt="compass" width="400"/>

Y luego, como es cícilico, le aplico una transformación sinusoidal.

In [130]:
directions = ["N", "NNE", "NE", "ENE", "E", "ESE", "SE", "SSE", "S", "SSW", "SW", "WSW", "W", "WNW", "NW", "NNW"]
dir_dict = dict([(dir, np.sin((2 * np.pi * i) / len(directions))) for i, dir in enumerate(directions)])
for col in ["WindGustDir", "WindDir9am", "WindDir3pm"]:
  data_frame.replace({col: dir_dict}, inplace=True)

### Lluvia
Ambas variables son binarias, así que las convierto a 0 y 1 para `"No"` y `"Yes"` respectivamente. 

In [131]:
for col in ["RainToday", "RainTomorrow"]:
  data_frame[col] = data_frame[col].eq('Yes').mul(1)

### `Location`
Veamos si son a lo sumo 20 (así según tu *rule of thumb* hago one hot encoding tranquilo)...

In [132]:
len(data_frame["Location"].unique())

44

... no. Veamos si puedo deshacerme de alguna etiqueta (por tener pocos valores)...

In [133]:
data_frame["Location"].value_counts()[-10:]

Tuggeranong    2316
Dartmoor       2294
Sydney         2259
Melbourne      2233
Williamtown    2195
Richmond       2048
Launceston     1538
Nhil           1518
Uluru          1446
Katherine       670
Name: Location, dtype: int64

... tampoco.

## TODO: Dividir en regiones y llorar.

## Extracción de features y target

In [134]:
features = data_frame.select_dtypes("number").drop(columns="RainTomorrow")
target = data_frame["RainTomorrow"]


# Visualización de datos